In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
# import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import utils, plot, cosmo
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# Initialize SAM model

In [ ]:
gsmf = holo.sam.GSMF_Schechter()        # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sam.GPF_Power_Law()          # Galaxy Pair Fraction         (GPF)
gmt = holo.sam.GMT_Power_Law()          # Galaxy Merger Time           (GMT)
mmbulge = holo.relations.MMBulge_MM13() # M-MBulge Relation            (MMB)

sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge)

# Sample (Discretize) Assuming GW-Driven Evolution

In [ ]:
# Choose observed GW-Frequency bins based on nyquist sampling
fobs = utils.nyquist_freqs(0.1*YR, 0.01*YR)
print(fobs)

In [ ]:
use_fobs = fobs
use_fobs = kale.utils.subdivide(fobs, 4)
print(fobs.size, use_fobs.size)
vals, weights, edges, dens = holo.sam.sample_sam_with_hardening(
        sam, holo.evolution.Hard_GW, fobs=use_fobs,
        sample_threshold=10.0, cut_below_mass=1e7, limit_merger_time=4*GYR,
)
print(weights.size)

In [ ]:
# dnum = sam._dnum
# print(dnum.shape)
# test = np.sum(dnum, axis=(0, 1, 2))
# print(test.shape)
# fig, ax = plot.figax()
# xx = sam._edges[-1]
# ax.plot(xx, test)
# plt.show()

log_edges = sam._log_edges
dnum = sam._dnum
axes = (1, 2)
log_edges = [log_edges[ii] for ii in range(len(log_edges)) if ii not in axes]
dnum = dnum.sum(axis=axes)

dnum = dnum[22:30, :7]
log_edges = [log_edges[0][22:30], log_edges[1][:7]]
print(dnum.shape)

fname = "kalepy_sample_outliers_error.npz"
np.savez(fname, dens=dnum, edges=log_edges)
print(f"Saved error example data to '{fname}'")

vals, weights = kale.sample_outliers(log_edges, dnum, 10.0)
fo = np.e ** vals[-1]
# ee = sam._edges[-1]
ee = np.e ** log_edges[-1]

fig, ax = plot.figax()
xx = ee
xx = kale.utils.subdivide(ee, 2, log=True)
print(weights.size, weights.sum(), weights.mean())
idx = ()
# idx = (weights == 1.0)
ax.hist(fo[idx], bins=xx, weights=weights[idx])

plt.show()

In [ ]:
fig, ax = plot.figax()
_ee = edges[-1] * YR
fo = vals[-1] * YR
# print(utils.stats(fo))
ee = kale.utils.subdivide(_ee, 3, log=True)
idx = ()
# idx = (weights != 1.0)
# idx = (weights == 1.0)
ax.hist(fo[idx], bins=ee, weights=weights[idx])
for e in _ee:
    ax.axvline(e, color='0.5', ls='--', alpha=0.2, zorder=1)
for e in ee:
    ax.axvline(e, color='r', ls='--', alpha=0.2, zorder=1)
plt.show()

In [ ]:
breaker()

# Calculate GWs from Samples

In [ ]:
gwf_freqs, gwf, gwb = holo.sam._gws_from_samples(vals, weights, fobs)

In [ ]:
# gwb_rough = sam.gwb(fobs, realize=33)
# gwb_smooth = sam.gwb(fobs, realize=False)

In [ ]:
fig, ax = plot.figax()

xx = kale.utils.midpoints(fobs) * YR   # [1/sec] ==> [1/yr]

amp = 10e-16
yy = amp * np.power(xx, -2/3)
ax.plot(xx, yy, 'k--', alpha=0.25)

# ff = np.sqrt(xx / np.diff(fobs*YR))
ff = 1.0

ax.plot(xx, gwb*ff, 'k-')

idx = (gwf > gwb)
xx = gwf_freqs * YR   # [1/sec] ==> [1/yr]
ax.scatter(xx[idx], (gwf*ff)[idx], color='r', s=20, alpha=0.5)
ax.scatter(xx[~idx], (gwf*ff)[~idx], edgecolor='r', facecolor='none', s=20, alpha=0.5)

# ax.plot(fobs*YR, gwb_smooth, 'b--')
# ax.plot(fobs*YR, np.median(gwb_rough, axis=-1), 'b:')
# ax.fill_between(fobs*YR, *np.percentile(gwb_rough, [25, 75], axis=-1), color='b', alpha=0.25)

plt.show()

# Use realistic hardening rate to sample population

In [ ]:
hard = holo.evolution.Fixed_Time.from_sam(sam, 2.0*GYR)